In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import matplotlib.dates as mdates
from matplotlib import animation, rc
from IPython.display import HTML

In [2]:
file_names = ['U_fft.pkl', 'V_fft.pkl', 'U_raw.pkl', 'V_raw.pkl', 'Time_fft.pkl', 
              'Time_II_fft.pkl', 'REA.pkl', 'SSC_rea.pkl', 'depth.pkl', 'wave_T_fft.pkl',
              'wave_H_fft.pkl','wave_dir_SC.pkl','wave_dt_stamp.pkl','wave_dt.pkl','C_R.pkl',
              'C_eo.pkl','C_sed.pkl','C_con.pkl','ssc_rouse.pkl','t_m_fft.pkl','ucr_fft.pkl',
              't_max_fft.pkl','Wind_U_spin.pkl','Wind_V_spin.pkl']

# Dicionário para armazenar os dados já processados de velocidadee outros
data = {}

# Loop através dos nomes dos arquivos e carrega os dados usando pickle
for file_name in file_names:
    with open(file_name, 'rb') as file:
        data[file_name.replace('.pkl', '')] = pickle.load(file)